# Benchmarking different quantum compilers


## Load benchmarks

In [ ]:
import os
import subprocess
from pathlib import Path

benchmark_files = list(Path(".").rglob("*.qasm"))

## Prepare compilers

In [ ]:
compilers = dict()

## QisKit

In [ ]:
from qiskit.compiler import transpile

In [ ]:
def optimize_qiskit(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    opt_qc = transpile(qc, optimization_level=3, approximation_degree=1.0,
                       basis_gates=['rz', 'rx', 'h', 't', 'tdag', 's', 'cx'])
    return opt_qc.qasm()


compilers['qiskit'] = optimize_qiskit

## PyZX

In [ ]:
import pyzx as zx

In [ ]:
def optimize_pyzx(qasm):
    c = zx.Circuit.from_qasm(qasm)
    c = c.to_graph()
    zx.simplify.full_reduce(c)
    c.normalize()
    c_opt = zx.extract_circuit(c.copy(), quiet=True, optimize_cnots=3).to_basic_gates()
    return c_opt.to_qasm()


compilers['pyzx'] = optimize_pyzx

## VOQC

To install voqc run
```shell
git submodule update --remote
cd pyvoqc
opam pin voqc https://github.com/inQWIRE/mlvoqc.git#mapping
./install.sh
```
Then copy pyvoqc/lib to your site packages pyvoqc folder

In [ ]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

In [ ]:
def optimize_voqc(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    vpm = voqc_pass_manager(post_opts=["optimize"])
    qc_opt = vpm.run(qc)
    return qc_opt.qasm()


compilers['voqc'] = optimize_voqc

## $\mid$tket$\rangle$

In [ ]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import SequencePass, RemoveRedundancies, FullPeepholeOptimise, auto_rebase_pass
from pytket.predicates import CompilationUnit
from pytket import OpType

In [ ]:
def optimize_tket(qasm):
    qc = circuit_from_qasm_str(qasm)
    gates = {OpType.Rz, OpType.Rx, OpType.H, OpType.T, OpType.Tdg, OpType.S, OpType.CX}
    rebase = auto_rebase_pass(gates)
    seqpass = SequencePass(
        [
            FullPeepholeOptimise(),
            RemoveRedundancies(),
            rebase,
        ])
    cu = CompilationUnit(qc)
    seqpass.apply(cu)
    qc_opt = cu.circuit
    return circuit_to_qasm_str(qc_opt)


compilers['tket'] = optimize_tket

## Staq

In [ ]:
import pystaq


In [ ]:
def optimize_staq(qasm):
    qc = pystaq.parse_str(qasm)
    pystaq.simplify(qc)
    pystaq.rotation_fold(qc)
    pystaq.simplify(qc)
    pystaq.cnot_resynth(qc)
    pystaq.simplify(qc)
    return str(qc)


compilers['staq'] = optimize_staq


# Run Benchmarks

## Gate counting

We count gates by finding them in the circuit representation.
Though, we assume a error corrected gate set, meaning we will synthesize any rotations that are not of form $c \times \frac{pi}{4}$ using Peter Sellinger's gridsynth (https://www.mathstat.dal.ca/~selinger/newsynth/)

In [ ]:
import subprocess


def gridsynth(theta):
    return subprocess.getoutput(f"./gridsynth {theta} -e 10e-8")

In [ ]:
from math import pi


def count_gates(qasm, gate_names):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in gate_names:
            count += 1
    return count


def approx(a, b):
    return abs(a - b) < 10e-4


def count_gates_synth(theta):
    synth = gridsynth(theta)
    return len(synth) - synth.count('W')


def count_t_synth(theta):
    synth = gridsynth(theta)
    return synth.count('T')


def count_rot_z_t(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += approx(pi / 4, abs(theta % (pi / 2)))
            else:
                count += count_t_synth(theta)

    return count


def count_t_gates(qasm):
    return count_gates(qasm, ['t', 'tdag']) + count_rot_z_t(qasm)


def count_cnot_gates(qasm):
    return count_gates(qasm, ['cx', 'cz'])


def count_total_gates(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += 1
            else:
                count += count_gates_synth(theta)
        else:
            count += 1
    return count


In [ ]:
from qiskit.circuit.library import MCMT

# Test
testing = False
if testing:
    qc = QuantumCircuit(2)
    qc.rz(0.132312, 0)
    qc.t(0)
    qc.h(1)
    qc.cnot(0, 1)
    qc.t(0)
    qc.cnot(0, 1)
    qc.h(0)
    qc.t(0)
    qc.t(1)
    qc.s(1)
    qc.cnot(1, 0)
    qc.rx(pi / 4, 1)

    print("Original")
    print(qc.draw())

    qasm = qc.qasm()

    for compiler_name in compilers:
        print(compiler_name)
        qasm_opt = compilers[compiler_name](qasm)
        qc_opt = QuantumCircuit.from_qasm_str(qasm_opt)
        print(qc_opt.draw())
        print(count_t_gates(qasm_opt))


In [ ]:
import time
import pickle

In [ ]:
# Create place to store files concerning optimiation and results
results_path = "results"
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [ ]:
def generate_result(qasm, time_taken=0):
    result = dict()
    result['qasm'] = qasm
    result['time'] = time_taken
    if not qasm == "":
        result['cnot_count'] = count_cnot_gates(qasm)
        result['t_count'] = count_t_gates(qasm)
        result['total_count'] = count_total_gates(qasm)
    return result

### Benchmark each compiler on its own and cache results
Will also generate baseline for non-optimized benchmarks

In [ ]:
import signal

class timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

In [ ]:
from qiskit.qasm import QasmError

In [ ]:
enable_large_benchmarks = False

In [ ]:
for benchmark_file in benchmark_files:
    if str(benchmark_file) == "temp_in.qasm" or str(benchmark_file) == "tmp.qasm":  # Ignore compiler artifacts
        continue
    path_no_slash = str(benchmark_file).replace("/", "-")
    print(benchmark_file)
    if (not enable_large_benchmarks) and os.path.getsize(benchmark_file) > 100 * 1000 : # 100 KB:
        print("Benchmark too large, skipping")
        continue
    try:
        benchmark = QuantumCircuit.from_qasm_file(benchmark_file) \
                              .decompose().qasm()  # Remove custom gate declarations since tket and voqc don't play nicely with them
    except QasmError as e:
        print(e)
        continue
    assert not benchmark == ""
    base_pkl_path = f"{results_path}/result_base_{path_no_slash}.pkl"
    if not os.path.exists(base_pkl_path):
        print("Establishing baseline")
        baseline = generate_result(benchmark)
        pickle.dump(baseline, open(base_pkl_path, 'wb'))
        print("Established baseline")
    else:
        print("Baseline already exists, continuing...")
    for compiler_name in compilers:
        print(f"Using {compiler_name} to optimize {benchmark_file}")
        pkl_path = f"{results_path}/result_{compiler_name}_{path_no_slash}.pkl"
        if os.path.exists(pkl_path):
            print("Found cached results file, skipping computation")
            continue
        try:
            with timeout(60 * 15):
                start = time.perf_counter()
                opt_qasm = compilers[compiler_name](benchmark)
                stop = time.perf_counter()
                time_taken = stop - start
                print(f"Completed in {time_taken} sec")
        except TimeoutError:
            print("Timed out after 15min")
            time_taken = -1
            opt_qasm = ""
        result = generate_result(opt_qasm, time_taken)
        pickle.dump(result, open(pkl_path, 'wb'))


### Use cached results to benchmark compiler juxtaposition

In [ ]:
# Benchmark individually
for compiler_name_1 in compilers:
    for compiler_name_2 in compilers:
        key = compiler_name_1 + '%' + compiler_name_2
        if compiler_name_1 == compiler_name_2:
            continue
        for benchmark_file in benchmark_files:
            if str(benchmark_file) == "temp_in.qasm" or str(benchmark_file) == "tmp.qasm":  # Ignore compiler artifacts
                continue
            path_no_slash = str(benchmark_file).replace("/", "-")
            print(benchmark_file)
            if (not enable_large_benchmarks) and os.path.getsize(benchmark_file) > 100 * 1000: # 100 KB:
                print("Benchmark too large, skipping")
                continue
            try:
                benchmark = QuantumCircuit.from_qasm_file(benchmark_file) \
                                      .decompose().qasm()  # Remove custom gate declarations since tket and voqc don't play nicely with them
            except QasmError as e:
                print(e)
                continue
            assert not benchmark == ""
            print(f"Using {compiler_name_2} to optimize {benchmark_file} previously optimized by {compiler_name_1}")
            pkl_path = f"{results_path}/result_{compiler_name_1}_{compiler_name_2}_{path_no_slash}.pkl"
            if os.path.exists(pkl_path):
                print("Found cached results file, skipping computation")
                continue
            prior_result_path = f"{results_path}/result_{compiler_name_1}_{path_no_slash}.pkl"
            if not os.path.exists(prior_result_path):
                print("Missing prior file, skipping")
                continue
            prior_result = pickle.load(open(prior_result_path, 'rb'))
            if prior_result['time'] == -1:
                print("Prior benchmark did not complete successfully, skipping")
            opt_benchmark = prior_result['qasm']
            assert not opt_benchmark == ""
            print(f"{compiler_name_1} + {compiler_name_2}")
            try:
                start = time.perf_counter()
                opt_qasm = compilers[compiler_name_2](opt_benchmark)
                stop = time.perf_counter()
                time_taken = stop - start
                print(f"Completed in {time_taken} sec")
            except Exception as e:
                print(e)
                opt_qasm = ''
                time_taken = -1
            result = generate_result(opt_qasm, time_taken)
            pickle.dump(result, open(pkl_path, 'wb'))